In [1]:
# Import the necessay System packages
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv

In [ ]:
# Now, using the ChatOpenAI
from langchain.chat_models import ChatOpenAI

# Create the OpenAI API Key
load_dotenv() # Takes environment variables from .env
KEY = os.getenv("OPENAI_API_KEY")

# Create ChatOpenAI Object
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [ ]:
# First, import the necessary packages
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [ ]:
# Define the PromptTemplate
TEMPLATE = """
Text: {text}
You are expert MCQ Maker. Given the above text, it is your job to create a \
quiz of {number} multiple choice questions for {subject} students in {tone} tone. \
Make sure the questions are not repeated and check all the questions to be conforming the text as well. \
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON {response_json}
"""

quiz_generation_prompt = PromptTemplate(
  input_variables=["text", "number", "subject", "tone", "response_json"],
  template=TEMPLATE
)

In [ ]:
# Creation of response_json
RESPONSE_JSON = {
    "1": {
	"mcq": "multiple choice question",
	"options": {
	    "a": "choice here",
	    "b": "choice here",
	    "c": "choice here",
	    "d": "choice here",
	},
	"correct": "correct answer",
    },
    "2": {
	"mcq": "multiple choice question",
	"options": {
	    "a": "choice here",
	    "b": "choice here",
	    "c": "choice here",
	    "d": "choice here",
	},
	"correct": "correct answer",
    },
    "3": {
	"mcq": "multiple choice question",
	"options": {
	    "a": "choice here",
	    "b": "choice here",
	    "c": "choice here",
	    "d": "choice here",
	},
	"correct": "correct answer",
    }
}

In [ ]:
# Using the LLMChain to connect the LLM and the PromptTemplate
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [ ]:
# For analyzing the correct answer. Created another template, then giving the template1 output as the template2 input.
EVALUATION_TEMPLATE = """
You are an expert English Grammarian and Writer. Given a Mutliple Choice Quiz for {subject} students. \
You need to evaluate the complexity of the question and give a complte analysis of the quiz. \
Only use at max 50 words for complexity if the quiz is not at per with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfects fits the student ability.
Quiz_MCQs: {quiz}

Check from an expert English Writer of the above quiz:
"""

quiz_evaluation_prompt = PromptTemplate(
  input_variables=["subject", "quiz"], 
  template=EVALUATION_TEMPLATE
)

In [ ]:
# Using again the LLMChain to connect the LLM and the PromptTemplate
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [ ]:
# Now, connecting both quiz_chain and review_chain using SequentialChain
generate_evaluate_chain = SequentialChain(
  chains=[quiz_chain, review_chain],
  input_variables=["text", "number", "subject", "tone", "response_json"],
  output_variables=["quiz", "review"],
  verbose=True
)

In [ ]:
# Use the data.txt file and conduct the quiz on it.
file_path = r"<absolute-path>\data.txt"

with open(file_path, 'r'), as file:
  TEXT = file.read()

# Now, python dictionary RESPONSE_JSON is converted into JSON. Using Json Serializer
# Serializing the Python Dictionary into a JSON-formatted string
RESPONSE = json.dumps(RESPONSE_JSON)

In [ ]:
# Final Step: Using get_openai_callback
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "EASY"

# https://python.langchain.com/v0.1/docs/modules/model_io/llms/token_usage_tracking/
# How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    final_response = generate_evaluate_chain({
	"text": TEXT,
	"number": NUMBER,
	"subject": SUBJECT,
	"tone": TONE,
	"response_json": RESPONSE
    })

In [ ]:
# Using get_openai_callback() method: We can use this method to track the tokens and the token usage.
# View the token usage.
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}") # Input Tokens
print(f"Completion Tokens: {cb.completion_tokens}") # Output Tokens
print(f"Total Cost (USD): ${cb.total_cost}")

In [ ]:
# View the Response
print(final_response)

# View the quiz questions and load back as python dictionary
quiz = final_response.quiz
quiz_dict = json.loads(quiz)
print(quiz_dict)

In [ ]:
# Now, formatting the dictionary
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " || ".join([
	f"{option}: {option_value}" for option, option_value in value["options"].items()
    ])
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

print(quiz_table_data)

In [ ]:
# Convert the quiz_table_data into a DataFrame and then convert into CSV file
quiz = pd.DataFrame(quiz_table_data)
quiz.to_csv("machinelearning.csv", index=False)